## Import library

In [345]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")

## Functions

In [346]:
def preprocess_raw_file(dataframe):
    dataframe['Month'] = dataframe['UPC'].copy()
    dataframe['UPC'] = pd.to_numeric(dataframe['UPC'], errors = 'coerce')
    dataframe['Month'] = pd.to_datetime(dataframe['Month'], errors = 'coerce')
    dataframe['Month'] = dataframe['Month'].fillna(method = 'ffill')
    dataframe = dataframe[dataframe['UPC'].notnull()]
    
    cols_to_align = ['UPC', 'Purch QTY', 'QTY', 'QTY, On Hand']
    for col in cols_to_align:
        dataframe[col] = dataframe[col].apply(lambda x: int(float(x)))

    dataframe['Month'] = dataframe['Month'].dt.to_period('M')

    return dataframe

In [347]:
def get_vendor_pivot_table(dataframe, vendor_name):
    df_vendor = dataframe[dataframe['Vendor'] == vendor_name]
    unique_months = df_vendor['Month'].sort_values().unique()
    df_pivot = pd.pivot_table(data = df_vendor, columns = 'Month', index = "Item Name", values = ['Purch QTY', 'QTY', 'QTY, On Hand'], \
                          aggfunc = np.sum)
    df_pivot = df_pivot.swaplevel(axis=1).sort_index(axis=1, level=0)
    df_pivot = df_pivot.fillna(0)

    return df_pivot, unique_months

In [348]:
def update_on_hand_qty(df_pivot, unique_months):
    for current_month in unique_months:
        if current_month == unique_months.min():
            df_pivot[(current_month, 'QTY, On Hand')] = df_pivot[(current_month, 'Purch QTY')] - df_pivot[(current_month, 'QTY')]
    
        else:
            previous_month = current_month - 1
            df_pivot[(current_month, 'QTY, On Hand')] = df_pivot[(previous_month, 'QTY, On Hand')] +\
            df_pivot[(current_month, 'Purch QTY')] - df_pivot[(current_month, 'QTY')]

    return df_pivot

In [349]:
def add_liq_percent(df_pivot, unique_months):

    for current_month in unique_months:
        qty = df_pivot[(current_month, 'QTY')]
        on_hand = df_pivot[(current_month, 'QTY, On Hand')]
        
        with np.errstate(divide='ignore', invalid='ignore'):
            liq_percent = qty / (qty + on_hand)
            liq_percent[np.isinf(liq_percent)] = 0
            liq_percent = np.nan_to_num(liq_percent, nan=0)  # handle NaNs if needed
    
        df_pivot[(current_month, 'LIQ %')] = liq_percent
        column_order = ['Purch QTY', 'QTY', 'QTY, On Hand', 'LIQ %']

        new_columns = []
        for month in unique_months:
            for col_name in column_order:
                if (month, col_name) in df_pivot.columns:
                    new_columns.append((month, col_name))
        df_pivot = df_pivot[new_columns]  # Reassign with ordered columns
        
    return df_pivot

In [350]:
def add_sale_qty(df_pivot, unique_months):

    for current_month in unique_months:
        if current_month == unique_months.min():
            df_pivot['Sale QTY'] = df_pivot[(current_month, 'QTY')]
        else:
            df_pivot['Sale QTY'] += df_pivot[(current_month, 'QTY')]
    return df_pivot

In [351]:
def add_shelf_life(df_pivot, unique_months):

    df_pivot['Shelf Life'] = 0
    for current_month in unique_months:
        df_pivot.loc[df_pivot[(current_month, 'QTY')] > 0, 'Shelf Life'] += 1
    return df_pivot

In [352]:
def add_avg_sales_per_month(df_pivot):

    df_pivot['Avg Sales/Mnth'] = df_pivot['Sale QTY'] / df_pivot['Shelf Life']
    df_pivot['Avg Sales/Mnth'] = df_pivot['Avg Sales/Mnth'].fillna(0)
    return df_pivot

In [353]:
def mround_excel(number, multiple):
    import math
    return multiple * round((number + 1e-10) / multiple) if number >= 0 else multiple * round((number - 1e-10) / multiple)

In [354]:
def add_bi_weekly_sale_forecasting(df_pivot):

    df_pivot['Bi-Weekly Sale Fsct'] = df_pivot['Avg Sales/Mnth'].apply(lambda x: mround_excel(x / 2, 1))

    return df_pivot

In [355]:
def add_safety_stock(df_pivot):

    df_pivot['Safety STK'] = df_pivot['Bi-Weekly Sale Fsct'].apply(lambda x: mround_excel(x * 0.5, 1))
    return df_pivot

In [356]:
def add_order(df_pivot, latest_month):
    df_pivot.loc[df_pivot['Safety STK'] + df_pivot['Bi-Weekly Sale Fsct'] > df_pivot[(latest_month, 'QTY, On Hand')], 'Order'] = \
    df_pivot['Safety STK'] + df_pivot['Bi-Weekly Sale Fsct'] - df_pivot[(latest_month, 'QTY, On Hand')]
    df_pivot.loc[df_pivot['Safety STK'] + df_pivot['Bi-Weekly Sale Fsct'] <= df_pivot[(latest_month, 'QTY, On Hand')], 'Order'] = 0
    return df_pivot

## Import file and preprocess

In [ ]:
df = pd.read_csv('./assets/example.csv', header = 8)

In [358]:
df.head(10)

,UPC,Item Name,GTIN,Size,Purch QTY,"Purch Amount,$",QTY,"Cost,$","QTY, On Hand","Cost, On Hand,$",...,Last Purch date,VIN,"Retail,$","Unit Cost,$","Unit Retail,$","Profit,$",GPM%,"Non-itemized Sales,$","Buydown by Sales,$","Total,$"
0,March 2024,NaN,NaN,NaN,NaN,NaN,NaN,16.00,164.80,35.00,...,0.00,0.00,887.00,13.38,25.34,418.74,47.21,0.00,0.0,887.00
1,69158402237,Stlth Type-C Black Metal,00691584022375,EA,5.0,37.45,0.0,0.00,0.00,0.00,...,08.30.2024,29990000001,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
2,69158402243,Stlth Type-C Device Green Metal,00691584022436,EA,5.0,37.45,0.0,0.00,0.00,0.00,...,03.04.2024,29990000001,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
3,5.05635E+11,IVG Punch,05056348088731,3000 Puffs,0.0,0.00,2.0,10.00,0.00,0.00,...,06.11.2024,29990000001,40.00,5.00,20.00,30.00,75.00,0.00,0.0,40.00
4,5.05662E+11,IVG Mango Ice,05056617511137,3000 Puffs,0.0,0.00,2.0,10.00,0.00,0.00,...,06.11.2024,29990000001,40.00,5.00,20.00,30.00,75.00,0.00,0.0,40.00
5,5.05662E+11,IVG Max Mango Peach Watermelon,05056617512035,5000 Puffs,0.0,0.00,5.0,84.95,0.00,0.00,...,10.17.2024,MMPW1020FDR,125.00,16.99,25.00,40.05,32.04,0.00,0.0,125.00
6,5.05662E+11,IVG Max Creamy Strawberry Vanilla Ice,05056617512158,5000 Puffs,0.0,0.00,10.0,149.83,0.00,0.00,...,08.30.2024,29990000001,250.00,14.98,25.00,100.17,40.07,0.00,0.0,250.00
7,5.05662E+11,IVG Max Strawberry Ice,05056617542049,5000 Puffs,6.0,89.90,0.0,0.00,0.00,0.00,...,10.04.2024,MSTI1020FDR,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
8,6.93764E+11,Elfbar BC Kiwi Passion Fruit Guava,06937643536433,5000 Puffs,0.0,0.00,5.0,68.49,0.00,0.00,...,NaN,NaN,135.00,13.70,27.00,66.51,49.27,0.00,0.0,135.00
9,6.93764E+11,Elfbar BC Grape Rush,06937643559364,5000 Puffs,0.0,0.00,9.0,119.99,0.00,0.00,...,07.25.2024,29990000001,243.00,13.33,27.00,123.01,50.62,0.00,0.0,243.00


In [359]:
df_processed = preprocess_raw_file(df)

## Filter Vendor

In [360]:
# Vapes
# 1. Rankin
    # a. G-Core
    # b. Fog Formula
    # c. Smoke Tokes
    # d. Maq Distribution
 
# 2. Walpole
    # a. Smoke Tokes
    # b. Maq Distribution
 
# 3. Fort Frances
    # a. G-Core
    # b. Fog Formula
    # c. Smoke Tokes
    # d. Maq Distribution
 
# 4. Silver Grizzly
    # a. G-Core
    # b. Fog Formula
    # c. Smoke Tokes
    # d. Maq Distribution
 
 
# Cannabis
# 1. Silver Grizzly

    # a. Proulx Commercial Growers
    # b. Cannabis Soda Co.
    # c. Native Flower

In [378]:
conda install streamlit

Jupyter detected...

Note: you may need to restart the kernel to use updated packages.



EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: C:\ProgramData\anaconda3




In [361]:
df_processed['Vendor'].unique()

array(['Smoke Arsenal', nan, 'Fog Formulas', 'Smoke Tokes',
       'G-Core Canada', 'Maq Distributors', 'Core-Mark'], dtype=object)

In [362]:
target_vendor = 'G-Core Canada'

## Create Pivot table

In [363]:
df_pivot, unique_months = get_vendor_pivot_table(df_processed, target_vendor)

In [364]:
df_pivot

Month                                                2024-05       \
                                                   Purch QTY  QTY   
Item Name                                                           
Caliburn G3 0.9OM Refil Pod 4PK                          0.0  1.0   
Flipper Blueberry Raspberry Ice-Blackberry Rasp...       0.0  0.0   
Flipper CRM BRU - Boston Mellow                          0.0  0.0   
Flipper Doctor Pepper Cherry-FNT Strawberry 20mg         0.0  0.0   
Flipper Extreme Mint-Wow Watermelon Ice                  0.0  0.0   
...                                                      ...  ...   
Rufpuf Righteous Blue Razz Ice                           0.0  0.0   
Rufpuf Twisted Melons                                    0.0  0.0   
Rufpuf Whacky Watermelon Strawberry Kiwi                 0.0  0.0   
Rufpuf Whacky Watermelon Strawberry Kiwi Rufpuf          0.0  0.0   
Uwell Caliburn G3 Black Pod System Kit 25W 900mAh        0.0  0.0   

Month                                                             2024-06  \
                                                   QTY, On Hand Purch QTY   
Item Name                                                                   
Caliburn G3 0.9OM Refil Pod 4PK                             0.0       0.0   
Flipper Blueberry Raspberry Ice-Blackberry Rasp...          0.0       5.0   
Flipper CRM BRU - Boston Mellow                             0.0      20.0   
Flipper Doctor Pepper Cherry-FNT Strawberry 20mg            0.0      25.0   
Flipper Extreme Mint-Wow Watermelon Ice                     0.0      10.0   
...                                                         ...       ...   
Rufpuf Righteous Blue Razz Ice                              0.0      50.0   
Rufpuf Twisted Melons                                       0.0      10.0   
Rufpuf Whacky Watermelon Strawberry Kiwi                    0.0       4.0   
Rufpuf Whacky Watermelon Strawberry Kiwi Rufpuf             0.0      30.0   
Uwell Caliburn G3 Black Pod System Kit 25W 900mAh           0.0       0.0   

Month                                                                  \
                                                     QTY QTY, On Hand   
Item Name                                                               
Caliburn G3 0.9OM Refil Pod 4PK                      0.0          0.0   
Flipper Blueberry Raspberry Ice-Blackberry Rasp...   0.0          0.0   
Flipper CRM BRU - Boston Mellow                      2.0          0.0   
Flipper Doctor Pepper Cherry-FNT Strawberry 20mg    14.0          0.0   
Flipper Extreme Mint-Wow Watermelon Ice              3.0          0.0   
...                                                  ...          ...   
Rufpuf Righteous Blue Razz Ice                      25.0          0.0   
Rufpuf Twisted Melons                                2.0          4.0   
Rufpuf Whacky Watermelon Strawberry Kiwi             0.0          0.0   
Rufpuf Whacky Watermelon Strawberry Kiwi Rufpuf     21.0          0.0   
Uwell Caliburn G3 Black Pod System Kit 25W 900mAh    0.0          0.0   

Month                                                2024-07        \
                                                   Purch QTY   QTY   
Item Name                                                            
Caliburn G3 0.9OM Refil Pod 4PK                          0.0   0.0   
Flipper Blueberry Raspberry Ice-Blackberry Rasp...       0.0   0.0   
Flipper CRM BRU - Boston Mellow                         20.0  11.0   
Flipper Doctor Pepper Cherry-FNT Strawberry 20mg        65.0  42.0   
Flipper Extreme Mint-Wow Watermelon Ice                 30.0  17.0   
...                                                      ...   ...   
Rufpuf Righteous Blue Razz Ice                          40.0  32.0   
Rufpuf Twisted Melons                                   11.0   3.0   
Rufpuf Whacky Watermelon Strawberry Kiwi                 1.0   5.0   
Rufpuf Whacky Watermelon Strawberry Kiwi Rufpuf         33.0  34.0   
Uwell Caliburn G3 Bla

### Update QTY, On Hand

In [365]:
df_pivot = update_on_hand_qty(df_pivot, unique_months)

### Add LIQ %

In [366]:
df_pivot = add_liq_percent(df_pivot, unique_months)

### Add Sale QTY

In [367]:
df_pivot = add_sale_qty(df_pivot, unique_months)

### Add Shelf Life

In [368]:
df_pivot = add_shelf_life(df_pivot, unique_months)

### Add Avg Sales/Mnth

In [369]:
df_pivot = add_avg_sales_per_month(df_pivot)

### Add Bi-Weekly Sale Fsct

In [370]:
df_pivot = add_bi_weekly_sale_forecasting(df_pivot)

### Add Safety STK

In [371]:
df_pivot = add_safety_stock(df_pivot)

### Order

In [372]:
latest_month = unique_months.max()

In [373]:
df_pivot = add_order(df_pivot, latest_month)

In [374]:
# sorting
df_pivot = df_pivot.sort_values(by = 'Avg Sales/Mnth', ascending = False)

In [376]:
# export
df_pivot.to_csv('df_pivot.csv')